<a href="https://colab.research.google.com/github/neenajoseph1008/gitbegin/blob/main/SCD2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q !wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
!pip install -q findspark
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Mounted at /content/drive


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SCD2").getOrCreate()

emp_add = spark.read.csv("/content/Emp.csv",inferSchema=True,header=True)
emp_add.show()

+------+-----+-------+------+------------+--------+
|emp_id| name|address|active|      eff_dt|end_date|
+------+-----+-------+------+------------+--------+
|     1|Neena|   Pune|     1|'2023-04-01'|  'null'|
|     2|Naina|Germany|     1|'2023-04-01'|  'null'|
+------+-----+-------+------+------------+--------+



In [6]:
incoming_emp_add = spark.read.csv("/content/Emp_12.txt",inferSchema=True,header=True)
incoming_emp_add.show()

+------+------+-------+
|emp_id|  name|address|
+------+------+-------+
|     1| Neena|   Pune|
|     2| Naina| Africa|
|     3|Akshay|Denmark|
+------+------+-------+



In [7]:
# Join incoming data with existing dimension data on employee_id
merged_data = incoming_emp_add.join(emp_add, "emp_id", "left_outer")

merged_data.show()

+------+------+-------+-----+-------+------+------------+--------+
|emp_id|  name|address| name|address|active|      eff_dt|end_date|
+------+------+-------+-----+-------+------+------------+--------+
|     1| Neena|   Pune|Neena|   Pune|     1|'2023-04-01'|  'null'|
|     2| Naina| Africa|Naina|Germany|     1|'2023-04-01'|  'null'|
|     3|Akshay|Denmark| null|   null|  null|        null|    null|
+------+------+-------+-----+-------+------+------------+--------+



In [ ]:
from pyspark.sql.functions import current_date, lit, col, when
# Define SCD2 logic
scd2_data = merged_data.select(
    col("employee_id"),
    when(col("existing_column") != col("incoming_column"), current_date()).otherwise(col("start_date")).alias("start_date"),
    lit(None).cast("date").alias("end_date"),
    col("incoming_column").alias("new_column")
)